In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from IPython.display import clear_output
from tqdm import tqdm
import copy
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

Cartesian = lambda x,y: np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
AddZ = lambda x: np.concatenate((x, np.zeros((x.shape[0], x.shape[1], 1))), axis=2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def Freq(Data = None):
  YantData = scipy.io.loadmat('/content/drive/MyDrive/AntData/YantData.mat')['YantData'].flatten()
  res = np.array([YantData[i].shape[0] for i in range(YantData.shape[0])])
  return res

def Cluster(Z,Y,plot = False):
    data = np.column_stack((Z, Y))
    centers = Cartesian(np.array([-25,-8,8,25]),np.array([-25,-8,8,25]))
    kmeans = KMeans(n_clusters=16, init=centers)
    kmeans.fit(data)

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_

    if plot:
      plt.figure(figsize=(12, 7))
      plt.title(f"Num of antennas: {Z.shape[0]}")
      plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='viridis')
      plt.scatter(centers[:, 0], centers[:, 1], marker='X',c='red', s=100, label='Centers')
      plt.grid()
      plt.show()

    return data, labels

def SmartPadding(Y,Z,i, flatten = True):
  Y, Z = Y[i][0].flatten(),Z[i][0].flatten()
  data, labels = Cluster(Z,Y)

  pad_data = None

  for l in range(16):
    labeled_data = np.vstack((data[labels == l], np.zeros((64 - np.sum(labels == l), 2))))
    pad_data = np.vstack((pad_data, labeled_data)) if pad_data is not None else labeled_data

  if flatten:
      return pad_data.flatten()
  else:
      return pad_data

def NoPadding(Z,Y,i):
    data = np.column_stack((np.squeeze(Z[i][0]), np.squeeze(Y[i][0])))
    return data.flatten()

def CreateData(mask = None):
    #For 3D data please: 1. Change to faltten = False, 2. Add the Add Z function in the end of the return, 3. Shout loudly: kulululu!
    YantData = scipy.io.loadmat('/content/drive/MyDrive/AntData/YantData.mat')['YantData']
    ZantData = scipy.io.loadmat('/content/drive/MyDrive/AntData/ZantData.mat')['ZantData']

    cost = scipy.io.loadmat('/content/drive/MyDrive/AntData/cost.mat')['cost'].squeeze()

    freq = Freq(YantData)
    if mask:
      YantData,ZantData,cost = YantData[freq == mask],ZantData[freq == mask],cost[freq == mask]
      data_list = [NoPadding(ZantData, YantData, i) for i in tqdm(range(ZantData.shape[0]))]
    else:
      data_list = [SmartPadding(ZantData, YantData, i, flatten = True) for i in tqdm(range(ZantData.shape[0]))]

    del YantData
    del ZantData

    return np.array(data_list), cost

def result(preds, reals):
  N = preds.shape[0]

  res = {'MPE':0,  '< 1%':0, '< 5%':0, '< 10%':0, '< 15%':0, '< 25%':0, '< 50%':0}


  for pred, real in zip(preds, reals):
    percentage_error = np.abs((pred - real)/ real) * 100
    try:
      res['MPE'] += (percentage_error / N)[0]
    except:
      res['MPE'] += (percentage_error / N)
    for i in [1,5,10,15,25,50]:
      res[f"< {i}%"] += 100 / N if percentage_error <= i else 0

  return res

result_table = lambda train_result, test_result: print(tabulate([['Train']+list(train_result.values()), ['Test']+list(test_result.values())], headers=list(train_result.keys()), tablefmt='pipe'))

In [ ]:
dict(zip(np.unique(Freq(), return_counts = True)[0],np.unique(Freq(), return_counts = True)[1]))

{800: 8400,
 804: 24000,
 808: 14400,
 812: 13200,
 816: 6000,
 820: 3600,
 824: 1200,
 828: 2400,
 832: 3600,
 836: 2400,
 840: 2400,
 844: 1200,
 848: 3600,
 852: 3600,
 856: 2400,
 860: 3600,
 864: 1200,
 868: 2400,
 872: 1200,
 884: 1200,
 888: 1200,
 1024: 1}

In [ ]:
mask = None
Data, cost = CreateData(mask = mask)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Data, cost, test_size=0.3, random_state=42)

del Data
del cost

100%|██████████| 103201/103201 [04:14<00:00, 404.72it/s]


In [ ]:
# Define input size
input_size = mask * 2 if mask is not None else 2048

# Create the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(input_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(input_size, activation='relu'),
    tf.keras.layers.Dense(input_size, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Get the training and testing loss from the history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the training and testing loss
plt.plot(train_loss[1:], label='Training Loss')
plt.plot(val_loss[1:], label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/500
2258/2258 [==============================] - 16s 6ms/step - loss: 4335447.5000 - val_loss: 3477691.5000
Epoch 2/500
2258/2258 [==============================] - 13s 6ms/step - loss: 1243807.8750 - val_loss: 1085053.6250
Epoch 3/500
2258/2258 [==============================] - 14s 6ms/step - loss: 965253.3750 - val_loss: 692461.1875
Epoch 4/500
2258/2258 [==============================] - 14s 6ms/step - loss: 860448.1250 - val_loss: 617188.1250
Epoch 5/500
2258/2258 [==============================] - 14s 6ms/step - loss: 802651.1875 - val_loss: 676576.5625
Epoch 6/500
2258/2258 [==============================] - 13s 6ms/step - loss: 750713.9375 - val_loss: 574150.3750
Epoch 7/500
2258/2258 [==============================] - 14s 6ms/step - loss: 734344.6875 - val_loss: 679892.0625
Epoch 8/500
2258/2258 [==============================] - 13s 6ms/step - loss: 735839.2500 - val_loss: 856494.6875
Epoch 9/500
2258/2258 [==============================] - 13s 6ms/step - loss: 684869

In [ ]:
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

train_result = result(train_predictions, y_train)
test_result = result(test_predictions, y_test)

#s1, s2 = X_train.shape[0]/(103200*0.7), X_test.shape[0]/(103200*0.3)

#train_result = {key: value * s1 for key, value in train_result.items()}
#test_result = {key: value * s2 for key, value in test_result.items()}

clear_output(wait = True)

result_table(train_result, test_result)

2 hidden layers network:

|       |     MPE |    < 1% |    < 5% |   < 10% |   < 15% |   < 25% |   < 50% |
|:------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Train | 51.1076 | 10.9635 | 46.1932 | 66.3054 | 74.2386 | 80.191  | 85.3904 |
| Test  | 52      | 10.8879 | 45.68   | 66.1994 | 74.5454 | 80.5723 | 85.7369 |

3 hidden layers network:

|       |     MPE |    < 1% |    < 5% |   < 10% |   < 15% |   < 25% |   < 50% |
|:------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Train | 34.1976 | 11.1157 | 46.704  | 67.2107 | 75.2049 | 81.5905 | 89.0421 |
| Test  | 34.8444 | 11.2109 | 46.5037 | 67.0424 | 75.3173 | 81.7932 | 89.3156 |

4 hidden layers network:

|       |     MPE |    < 1% |    < 5% |   < 10% |   < 15% |   < 25% |   < 50% |
|:------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Train | 69.0754 | 10.753  | 46.1822 | 65.8472 | 73.0828 | 78.2807 | 83.7583 |
| Test  | 70.3839 | 10.9073 | 45.9029 | 65.7698 | 73.2567 | 78.6312 | 83.9702 |

Ensemble of 5 3 layers networks:

|       |     MPE |    < 1% |    < 5% |   < 10% |   < 15% |   < 25% |   < 50% |
|:------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Train | 25.4391 | 11.0783 | 45.9953 | 66.7539 | 75.9372 | 84.2068 | 93.5797 |
| Test  | 21.5248 | 11.1463 | 45.764  | 66.4029 | 75.8373 | 84.1639 | 93.4789 |

Model (2 hidden layers) per number of antennas:

|       |     MPE |    < 1% |    < 5% |   < 10% |   < 15% |   < 25% |   < 50% |
|:------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Train | 7.86155 | 12.6315 | 52.0792 | 73.2309 | 83.8566 | 95.0761 |  99.546 |
| Test  | 8.70262 | 10.9367 | 47.4031 | 70.407  | 81.6182 | 93.7435 |  99.241 |